In [20]:
import pandas as pd
import numpy as np
import os
import sys

from pathlib import Path
path = Path("../").resolve()  
sys.path.append(str(path))

import utils.metric as metric

In [21]:
df = pd.read_csv('../predict/squall_classifier_test1.csv')
df.head()

,id,tbl,question,answer,acc_tableqa,ans_tableqa,acc_text_to_sql,ans_text_to_sql,query_pred,query_fuzzy,...,truncated_tableqa,input_tokens,hq_overlap,complex_cols,num_pro_col,tqa_is_abb,tts_is_abb,tqa_is_sub,tts_is_sub,partial_cells
0,nu-0,203_733,which country had the most cyclists finish wit...,Italy,1,italy,0,ita,select 5_cyclist_second from w group by 5_cycl...,select c2_second from w group by c2_second ord...,...,0,<s> which country had the most cyclists finish...,1,NaN,2,0,0,0,0,0
1,nu-1,204_149,how many people were murdered in 1940\\/41?,"100,000",1,"100,000",1,100000.0,select 6_1940/41_number from w where 3_descrip...,select c3_number from w where c1 ='murdered',...,0,<s> how many people were murdered in 1940\\/41...,0,c1,1,0,0,0,0,0
2,nu-2,203_435,how long did it take for the new york american...,17 years,0,2 years,0,22,select max ( 3_year_maximum_year ) - min ( 3_y...,select max ( c1_maximum_year ) - min ( c1_mini...,...,0,<s> how long did it take for the new york amer...,1,NaN,2,0,0,0,0,0
3,nu-3,204_803,alfie 's birthday party aired on january 19 . ...,"January 26, 1995",1,"january 26, 1995",1,"january 26, 1995",select 9_original_air_date from w where 1_id =...,select c5 from w where id = ( select id from w...,...,0,<s> alfie's birthday party aired on january 19...,0,c4,0,1,1,1,1,0
4,nu-4,204_272,what is the number of 1st place finishes acros...,17,0,18,1,17,select count ( * ) from w where 13_placing_num...,select count ( * ) from w where c6_number = 1,...,0,<s> what is the number of 1st place finishes a...,1,NaN,1,0,0,0,0,0


In [28]:
df2 = df.copy()
df2['acc_tableqa'] = df2.apply(lambda x: 1 if metric.check_match(str(x['ans_tableqa']), str(x['answer'])) else 0, axis=1)
df2['acc_text_to_sql'] = df2.apply(lambda x: 1 if metric.check_match(str(x['ans_text_to_sql']), str(x['answer'])) else 0, axis=1)
df2.head()

,id,tbl,question,answer,acc_tableqa,ans_tableqa,acc_text_to_sql,ans_text_to_sql,query_pred,query_fuzzy,...,truncated_tableqa,input_tokens,hq_overlap,complex_cols,num_pro_col,tqa_is_abb,tts_is_abb,tqa_is_sub,tts_is_sub,partial_cells
0,nu-0,203_733,which country had the most cyclists finish wit...,Italy,1,italy,1,ita,select 5_cyclist_second from w group by 5_cycl...,select c2_second from w group by c2_second ord...,...,0,<s> which country had the most cyclists finish...,1,NaN,2,0,0,0,0,0
1,nu-1,204_149,how many people were murdered in 1940\\/41?,"100,000",1,"100,000",1,100000.0,select 6_1940/41_number from w where 3_descrip...,select c3_number from w where c1 ='murdered',...,0,<s> how many people were murdered in 1940\\/41...,0,c1,1,0,0,0,0,0
2,nu-2,203_435,how long did it take for the new york american...,17 years,0,2 years,0,22,select max ( 3_year_maximum_year ) - min ( 3_y...,select max ( c1_maximum_year ) - min ( c1_mini...,...,0,<s> how long did it take for the new york amer...,1,NaN,2,0,0,0,0,0
3,nu-3,204_803,alfie 's birthday party aired on january 19 . ...,"January 26, 1995",1,"january 26, 1995",1,"january 26, 1995",select 9_original_air_date from w where 1_id =...,select c5 from w where id = ( select id from w...,...,0,<s> alfie's birthday party aired on january 19...,0,c4,0,1,1,1,1,0
4,nu-4,204_272,what is the number of 1st place finishes acros...,17,0,18,1,17,select count ( * ) from w where 13_placing_num...,select count ( * ) from w where c6_number = 1,...,0,<s> what is the number of 1st place finishes a...,1,NaN,1,0,0,0,0,0


In [29]:
print('TableQA Accuracy:', df['acc_tableqa'].mean())
print('Text-to-SQL Accuracy:', df['acc_text_to_sql'].mean())

print('TableQA Accuracy:', df2['acc_tableqa'].mean())
print('Text-to-SQL Accuracy:', df2['acc_text_to_sql'].mean())



TableQA Accuracy: 0.6261510128913443
Text-to-SQL Accuracy: 0.6466390423572744
TableQA Accuracy: 0.6519337016574586
Text-to-SQL Accuracy: 0.6832412523020258


In [30]:
df2['gpt_score'] = 0.0
df2.to_csv('../predict/squall_classifier_test1_custom.csv', index=False)

In [27]:
df3 = pd.read_csv('../llm/squall/squall_classifier_test1_custom_gpt.csv')
df3.head()

,id,tbl,question,answer,acc_tableqa,ans_tableqa,acc_text_to_sql,ans_text_to_sql,query_pred,query_fuzzy,...,tqa_is_abb,tts_is_abb,tqa_is_sub,tts_is_sub,partial_cells,gpt_score,similarity,entityAlign,numberAlign,comparison
0,nu-0,203_733,which country had the most cyclists finish wit...,Italy,True,italy,True,ita,select 5_cyclist_second from w group by 5_cycl...,select c2_second from w group by c2_second ord...,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
1,nu-1,204_149,how many people were murdered in 1940\\/41?,"100,000",True,"100,000",True,100000.0,select 6_1940/41_number from w where 3_descrip...,select c3_number from w where c1 ='murdered',...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,nu-2,203_435,how long did it take for the new york american...,17 years,False,2 years,False,22,select max ( 3_year_maximum_year ) - min ( 3_y...,select max ( c1_maximum_year ) - min ( c1_mini...,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,nu-3,204_803,alfie 's birthday party aired on january 19 . ...,"January 26, 1995",True,"january 26, 1995",True,"january 26, 1995",select 9_original_air_date from w where 1_id =...,select c5 from w where id = ( select id from w...,...,1,1,1,1,0,0,NaN,NaN,NaN,NaN
4,nu-4,204_272,what is the number of 1st place finishes acros...,17,False,18,True,17,select count ( * ) from w where 13_placing_num...,select count ( * ) from w where c6_number = 1,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
